# `jupyterlite-pyodide-lock`

Many features of the `jupyterlite-pyodide-lock` can be configured along with
other JupyterLite features in your `jupyter_lite_config.json`.

In [ ]:
import json

import traitlets
from IPython.display import HTML
from jupyterlite_core.trait_types import TypedTuple
from nbconvert.filters.markdown_mistune import markdown2html_mistune
from traitlets.config import Configurable

In [ ]:
TArrayish = traitlets.List | traitlets.Tuple | TypedTuple
TNumberish = traitlets.CInt | traitlets.Int | traitlets.Float

In [ ]:
def markdown(md: str) -> HTML:
    """Generate some HTML."""
    return HTML(markdown2html_mistune(md))

In [ ]:
def trait_to_json_type(trait: traitlets.TraitType) -> list[str]:
    """Extract a simplified JSON type from a trait."""
    json_type = ""
    item_type = ""
    if isinstance(trait, TArrayish):
        json_type = "array"
        item_type = trait_to_json_type(trait._trait)[0]  # noqa: SLF001
    elif isinstance(trait, traitlets.Enum):
        return "string", " <br/> ".join(f"`{v}`" for v in trait.values)
    elif isinstance(trait, traitlets.Bool):
        json_type = "boolean"
    elif isinstance(trait, traitlets.Unicode):
        json_type = "string"
    elif isinstance(trait, TNumberish):
        json_type = "number"
    elif isinstance(trait, traitlets.Dict):
        json_type = "object"
    else:
        msg = f"unexpected trait {trait}"
        raise ValueError(msg)
    return json_type, item_type

In [ ]:
def trait_to_json_type_and_default(trait: traitlets.TraitType) -> list[str]:
    """Extract a simplified JSON type and default from a trait."""
    json_type, item_type = trait_to_json_type(trait)

    default_value = trait.default_value

    if isinstance(trait, traitlets.Dict):
        default_value = {}
    elif trait.default_value is traitlets.Undefined:
        default_value = None
        if isinstance(trait, TArrayish):
            default_value = []
    return [
        json_type or "",
        item_type or "",
        ("" if default_value is None else f"`{json.dumps(default_value)}`"),
    ]

In [ ]:
def config_table(importable: str) -> str:
    """Generate a directive for a filtered configurable."""
    current = __import__(importable.rsplit(".", 1)[0])
    for bit in importable.split(".")[1:]:
        current = getattr(current, bit)
    if not issubclass(current, Configurable):
        msg = f"{importable} is not a Configurable"
        raise ValueError(msg)
    lines = [
        f"### {current.__name__}",
        "| name | help | type | [items] | default |",
        "|:-|:-|:-:|:-:|:-|",
    ]
    traits = current._traits  # noqa: SLF001
    configurables = {
        trait_name: trait
        for trait_name, trait in sorted(traits.items())
        if trait.metadata.get("config")
    }
    for trait_name, trait in configurables.items():
        line = [
            f"`{trait_name}`",
            f"{trait.help}",
            *trait_to_json_type_and_default(trait),
        ]
        lines += [f"""| {" | ".join(line)} |"""]
    return "\n".join(lines)

## Addons

The `PyodideLockAddon` provides the main tasks for working with lockfile requirements.

In [ ]:
if __name__ == "__main__":
    display(
        markdown(config_table("jupyterlite_pyodide_lock.addons.lock.PyodideLockAddon"))
    )

## Lockers

The `BaseLocker` subclasses provide an WebAssembly environment in which to run `micropip.freeze`.

In [ ]:
if __name__ == "__main__":
    display(
        markdown(config_table("jupyterlite_pyodide_lock.lockers.browser.BrowserLocker"))
    )

## Appendix: CI

In some continuous integration settings such as GitHub Actions, `chromium`-like browsers may fail for mysterious reasons.

## Chromium Logs

A first step is gathering better logs:

```json
{
  "BrowserLocker": {
    "extra_browser_argv": [
      "--enable-logging=stderr", 
      "--v=1"
    ]
  }
}
```

This generates an _enormous_ amount of output, but will help at least find what to search for in the _thousands_ of `chromium` flags.

## Chromium in Containers

Some container-based builds are fixed by:

```json
{
  "BrowserLocker": {
    "extra_browser_argv": [
      "--remote-debugging-port=0",
      "--disable-dev-shm-usage",
      "--disable-gpu"
    ]
  }
}
```

## Chromium Sandbox

A (sometimes unavoidable) step is to disable the browser sandbox altogether:

```json
{
  "BrowserLocker": {
    "extra_browser_argv": [
      "--no-sandbox"
    ]
  }
}
```